In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.saved_model import loader

import utils
import data_mappers
import nn_models
tf.logging.set_verbosity(tf.logging.WARN)
tf.__version__

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use(['seaborn-paper'])

In [ ]:
our_recs = sorted(utils.enumerate_recording_paths('../dataset/raw/*record*'))
our_recs += sorted(utils.enumerate_recording_paths('../dataset/raw/benni*/*record*'))
kev_recs = sorted(utils.enumerate_recording_paths('/Users/pabloprietz/Dropbox/DCAITI_sharedfolder/Recordings/Kevin/rec*'))
del kev_recs[:4]  # remove recordings with different sampling rate

# All cases

## Export windows

In [ ]:
for single_rec in our_recs + kev_recs:
    print(f'Exporting {single_rec}')
    feeder = data_mappers.Window_Creator(rec_paths=[single_rec], train_test_split=1., window_size=120, subsample=1)
    windows = feeder.accelerometer
    labels = feeder.labels
    np.savez_compressed(os.path.join(single_rec, 'window_export'), input=windows, labels=labels)

## Predict windows

In [ ]:
mlp_120_1 = 'saved_models/20-20_120_1_0.01_0.95/1519831256'
cnn_120_1 = 'saved_models/no_maxpool_10-5--10-5_120_1_0.01_0.95/1519994594'

In [ ]:
all_recs = our_recs #+ kev_recs
saved_model_dir = cnn_120_1
tag_set = ['serve']

with tf.Session(graph=tf.Graph()) as sess:
    print(f'Loading model {saved_model_dir}')
    loader.load(sess, tag_set, saved_model_dir)
    graph_input = sess.graph.get_operation_by_name('accelerometer_input').values()[0]
    graph_output = sess.graph.get_operation_by_name('softmax_tensor').values()[0]

    for single_rec in all_recs:
        print(f'Predicting {single_rec}')
        data = np.load(os.path.join(single_rec, 'window_export.npz'))
        prediction = sess.run(graph_output, feed_dict={graph_input: data['input']})
        np.savez_compressed(os.path.join(single_rec, 'window_export.npz'), prediction=prediction, input=data['input'], labels=data['labels'])

## Visualize all predictions

In [ ]:
all_recs = our_recs + kev_recs
cols = 2
rows = len(all_recs) // cols + (0 if len(all_recs) % cols == 0 else 1)
from string import ascii_uppercase

for i, single_rec in enumerate(all_recs):
    plt.figure(figsize=(10, 3))
#     plt.subplot(rows, cols, i + 1)
    data = np.load(os.path.join(single_rec, 'window_export.npz'))
    windows = data['input']
    labels = data['labels']
    prediction = data['prediction']
    
    for d in [2]: # range(windows.shape[2]):
        plt.plot(windows[:, windows.shape[1] // 2, d], label=('x', 'y', 'z')[d])

#     for cls in range(prediction.shape[1]):
#         plt.plot(prediction[:, cls], label=('not walking', 'walking')[cls])
    plt.plot(np.argmax(prediction, axis=1), label='prediction')
        
    plt.plot(labels, label='labels')
    plt.plot([0, 120], [1.1, 1.1], label='window width')
    plt.legend()
    plt.title(f'Recording {ascii_uppercase[i]}')
    plt.xlabel('Window Index')
    plt.ylabel('Normalized acceleration')
    plt.ylim([-0.1, 1.2])
    plt.tight_layout()
    plt.savefig(f'figures/all_windows_prediction_rec_{ascii_uppercase[i]}.png')
    plt.show()
#     plt.close()

# Edge cases
Edge cases are windows whose ground truth sample labels have a majorty of less than 2/3, i.e. 0.33 < `raw_labels` < .67

## Filter and save edge cases

In [ ]:
for i, single_rec in enumerate(our_recs + kev_recs):
    feeder = data_mappers.Window_Creator(rec_paths=[single_rec], train_test_split=1., window_size=120, subsample=1)
    raw_labels = feeder.raw_labels
        
    edge_cases = np.flatnonzero(np.logical_and(0.33 < raw_labels, raw_labels < 0.67))
    edge_case_windows = feeder.accelerometer[edge_cases]
    edge_case_labels = raw_labels[edge_cases]
    print(f'Found {edge_cases.shape[0]} edge cases in {single_rec}')
    np.savez_compressed(os.path.join(single_rec, 'edge_cases'), input=edge_case_windows, labels=edge_case_labels)

## Predict edge cases

In [ ]:
all_recs = our_recs + kev_recs
saved_model_dir = cnn_120_1
tag_set = ['serve']

with tf.Session(graph=tf.Graph()) as sess:
    print(f'Loading model {saved_model_dir}')
    loader.load(sess, tag_set, saved_model_dir)
    graph_input = sess.graph.get_operation_by_name('accelerometer_input').values()[0]
    graph_output = sess.graph.get_operation_by_name('softmax_tensor').values()[0]

    for i, single_rec in enumerate(our_recs + kev_recs):
        print(f'Predicting edge cases in {single_rec}')
        data = np.load(os.path.join(single_rec, 'edge_cases.npz'))
        prediction = sess.run(graph_output, feed_dict={graph_input: data['input']})
        np.savez_compressed(os.path.join(single_rec, 'edge_cases.npz'), prediction=prediction, input=data['input'], labels=data['labels'])

## Visualize edge case predictions

In [ ]:
for i, single_rec in enumerate(our_recs + kev_recs):
    data = np.load(os.path.join(single_rec, 'edge_cases.npz'))
    
    plt.figure(figsize=(20, 3))
    plt.plot([0, data['labels'].shape[0]], [.5, .5])
    plt.plot(data['labels'], label='raw labels')
    plt.plot(data['prediction'][:, 1], label='prediction')
    plt.legend()
    plt.ylim([-.1, 1.1])
    plt.xlabel('Window Index')
    plt.ylabel('Label Mean')
    plt.title(f'Recording {ascii_uppercase[i]}')
    plt.tight_layout()

    plt.savefig(f'figures/low_label_conf_rec_{ascii_uppercase[i]}.png')
    plt.show()
#     plt.close()

In [ ]:
1